# Simultaneous generation-classification using LSTM (only classification)
Implementation of the paper: Simultaneous Generation-classification using lstm 

Authors: Daniel L. Marino, Kasun Amarasinghe, Milos Manic


In [1]:
import os
import numpy as np
import random
import tensorflow as tf
import pickle
import math
import matplotlib.pyplot as plt
from six.moves import range

from twodlearn.tf_lib.Feedforward import LinearLayer
from twodlearn.tf_lib.Recurrent import *

import sys
working_dir= os.getcwd()

print('Working directory:', working_dir)

Working directory: /home/marinodl/projects/co_generation_classification/sentiment_analysis_imdb


#### Configuration Variables

In [2]:
allow_valid= True
allow_test= True
Valid_Percentage= 0.1 # i.e. 10% of the data will be used for validation

pos_net_name= '_pos'
neg_net_name= '_neg'

activation_function='tanh'
num_nodes = [300, 300] #num_nodes: Nodes for the LSTM cell
alpha = 10.0 #0.1 #0.1
beta = 0.1 #0.01 #10000.01
lambda_w = 0.00001

dropout_cons = 0.8

Allow_Bias= False 

learning_rate= 0.005      # 0.001
grad_clip_thresh= 1.1       # 0.00001

current_run= 1
batch_size= 64 #64
num_unrollings= 64 #100

batch_size_val= 64 #len(valid_text_pos)/num_unrollings_val #500
num_unrollings_val= num_unrollings #100

batch_size_test= 64 
num_unrollings_test= num_unrollings #100

comment='_noDropout_LcLpLcp_64unrol_standarloss'

In [3]:
model_version = 'L'+str(len(num_nodes))
print("num_nodes:",num_nodes)
print("alpha:",alpha, ", beta:",beta,", lambda_w:", lambda_w)
print("num_unrollings:",num_unrollings, ", batch_size:",batch_size,", batch_size_val:", batch_size_val)
print("learning_rate:", learning_rate, 'grad_clip_thresh:', grad_clip_thresh)

num_nodes: [300, 300]
alpha: 10.0 , beta: 0.1 , lambda_w: 1e-05
num_unrollings: 64 , batch_size: 64 , batch_size_val: 64
learning_rate: 0.005 grad_clip_thresh: 1.1


# 1. Load training dataset

In [4]:
vc= pickle.load( open( "imdb_vc.pkl", "rb" ) )
num_inputs=  vc.vocabulary_size
num_outputs= vc.vocabulary_size

dataset= pickle.load( open( "imdb_dataset.pkl", "rb" ) )

# set batch_size and number of unrolligns
dataset.train.set_batch_and_unrollings(batch_size, num_unrollings)
dataset.valid.set_batch_and_unrollings(batch_size_val, num_unrollings_val)
dataset.test.set_batch_and_unrollings(batch_size_test, num_unrollings_test)

# print a sample of the dataset
train_x, train_y= dataset.train.next_batch()
print(len(train_x))
print(train_y.shape)
print(train_x[0].shape)

print(vc.keys2text([np.argmax(train_x[i][0,:], 0) for i in range(len(train_x))]))
print(train_y[0])

print(vc.keys2text([np.argmax(train_x[i][50,:], 0) for i in range(len(train_x))]))
print(train_y[50])

valid_x, valid_y= dataset.valid.next_batch()

65
(64, 1)
(64, 3001)
side of the countri like in new york citi or someth and ha absolut noth to do with the event go on at lake onli like twenti minut later the pop up again in and there s an chase through the wood end in the s the absurd littl detail in the lake monster are too numer to mention for exampl thi is probabl the 
[ 0.]
expens restaur if thi sound implaus to you you should probabl avoid thi film it act as my to disbelief and i had a lot more fun for it chase down is sam lee a young cop who like to peopl around almost as much as he like to smoke walk a fine line that ha intern affair investig him and hi father a legendari 
[ 1.]


In [5]:
print('# inputs:',num_inputs)

print('pos train length:',dataset.train.batch_generators[0]._text_size )
print('neg train length:',dataset.train.batch_generators[1]._text_size )

print('pos valid length:',dataset.valid.batch_generators[0]._text_size )
print('neg valid length:',dataset.valid.batch_generators[1]._text_size )

print('pos test length:',dataset.test.batch_generators[0]._text_size )
print('neg test length:',dataset.test.batch_generators[1]._text_size )

# inputs: 3001
pos train length: 2422173
neg train length: 2422173
pos valid length: 269130
neg valid length: 269130
pos test length: 2633361
neg test length: 2633361


# 2. Auxiliary functions

In [6]:
def classification_error(predictions, labels):
    """ number of samples wrongly classified  """
    return np.sum( np.not_equal( np.greater(predictions, 0.5), 
                                 np.greater(labels, 0.5)) )/labels.shape[0]

def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10 # this is to prevent that log() returns minus infinity
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vc.vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vc.vocabulary_size])
    return b/np.sum(b, 1)[:,None]

# 3. Model for positive and negative reviews

In [7]:
class myLstmNet(LstmNet):
    def get_extra_inputs(self, i, h_list, state_list):
        #print('OK:', len(h_list))
        return i
    
    def evaluate_final_output(self, outputs_list, inputs_list, h_list ):
        ''' Calculates the final output of the neural network, usually it is just a linear transformation
            - outputs_list: list with the outputs from the last lstm cell
            - inputs_list: list of inputs to the network
            - h_list: list with all hidden outputs from all the cells
        '''
        all_hidden = list()
        
        for t in h_list: # go trough each time step
            all_hidden.append( tf.concat(1,t) )
        return self.out_layer.evaluate(tf.concat(0, all_hidden))  
            
    
if len(num_nodes)>1:
    n_extra= [num_inputs for i in range(len(num_nodes)+1)]
    n_extra[0]= 0
    n_extra[-1]= sum(num_nodes) - num_nodes[-1]
else:
    n_extra= [0,0]

In [ ]:
class ModelSetup:
    
    def __init__( self, pos_net, neg_net, w, b, batch_size, num_unrollings, drop_prob_list, name=''):
    
        # 1. Create placeholders for inputs 
        self.X = list()
        for iaux in range(num_unrollings + 1):
            self.X.append(tf.placeholder(tf.float32, shape=[batch_size, vc.vocabulary_size], 
                                             name= name+'X_i'+str(iaux)+'_All'))
        aux_inputs = self.X[:num_unrollings]
        aux_labels = self.X[1:]  # inputs shifted by one time step.

        # Create a list for store the placeholders for the labels
        self.labels = tf.placeholder(tf.float32, shape=[batch_size, 1])
       

        # -------------------- unrolling of the network --------------------------- # 
        self.pos_unroll, _= pos_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   )

        self.neg_unroll, _= neg_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   ) 


        # Classifier.
        # error_per_sample is a vector, its shape is changed to have each unrolling in separate columns
        y_pos= tf.reshape(self.pos_unroll.y, [-1, batch_size, num_outputs])
        y_neg= tf.reshape(self.neg_unroll.y, [-1, batch_size, num_outputs])
        
        y_pos_mean= tf.reduce_mean(y_pos, reduction_indices=0)
        y_neg_mean= tf.reduce_mean(y_neg, reduction_indices=0)
        
        output_mean= tf.concat(1, [ y_pos_mean, y_neg_mean ])

        self.logits = tf.nn.xw_plus_b( output_mean , w, b )
        
        self.error_per_sample= tf.nn.sigmoid_cross_entropy_with_logits( self.logits, self.labels )
        
        '''
        output_pos= tf.reshape(self.pos_unroll.error_per_sample,[num_unrollings,batch_size])  
        output_neg= tf.reshape(self.neg_unroll.error_per_sample,[num_unrollings,batch_size]) 

        output_pos_mean= tf.reduce_mean(output_pos, reduction_indices= 0) 
        output_neg_mean= tf.reduce_mean(output_neg, reduction_indices= 0) 

        output_mean= tf.transpose(tf.pack( [ output_pos_mean, output_neg_mean ]))


        self.logits = tf.nn.xw_plus_b( output_mean , w, b )


        self.error_per_sample= tf.nn.sigmoid_cross_entropy_with_logits( self.logits, self.labels )
        '''
        
        # regularization
        l2_c= tf.nn.l2_loss(w)
        
        # loss
        self.loss = tf.reduce_mean( self.error_per_sample ) 

In [ ]:

graph = tf.Graph()
with graph.as_default():
    # For dropout
    drop_prob = tf.placeholder(tf.float32)
    drop_prob_list = [ drop_prob for i in range(len(num_nodes)+1)]
    drop_prob_list[0]= None
     
    # 1. Define positive and negative neural networks
    pos_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= pos_net_name)
        
    neg_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= neg_net_name)
    
    # Classifier weights and biases.
    #w = tf.Variable(tf.truncated_normal([2, 1], -0.1, 0.1), name=('w_class')) # unconstrained w
    #w = tf.Variable(tf.constant( [[-2.0], [2.0]]), name=('w_class'), trainable=False) # fixed w
    
    #w = tf.Variable(tf.truncated_normal([1, 1], 0.3, 0.4), name=('w_class')) # unconstrained w
    w = tf.Variable(tf.constant( [[0.3]] ), name=('w_class')) # fixed w
    w = tf.concat(0, [-w, w])
    
    b = tf.Variable(tf.zeros([1]), name=('b_class'), trainable=Allow_Bias)
    
    
    
    # 2. Define unrolling for training
    train= ModelSetup( pos_net, neg_net, w, b, batch_size, num_unrollings, 
                       drop_prob_list, 
                       name='train_')
       
    # 3. Define unrolling for validation
    if allow_valid:
        valid= ModelSetup( pos_net, neg_net, w, b, batch_size_val, num_unrollings_val, 
                           drop_prob_list= [None for dummy in range(len(num_nodes))], 
                           name='valid_')
        
    # 4. Define unrolling for testing
    if allow_test:
        test= ModelSetup( pos_net, neg_net, w, b, batch_size_test, num_unrollings_test, 
                          drop_prob_list= [None for dummy in range(len(num_nodes))], 
                          name='test_')
    
    # 5. Define unrolling for testing generator
    pos_gen_test, _= pos_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    neg_gen_test, _= neg_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    
          
    
    # 6. Define optimizer    
    # 1. specify the optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate) # ADAM 0.001
    
    # 2. get the gradients and variables
    # grads_and_vars is a list of tuples (gradient, variable). 
    grads_and_vars = optimizer.compute_gradients(train.loss) 
    gradients, v = zip(*grads_and_vars)
    
    # 3. process the gradients
    gradients, _ = tf.clip_by_global_norm(gradients, grad_clip_thresh)  #1.25 #0.025 #0.001(last used)
    # 4. apply the gradients to the optimization procedure
    optimizer = optimizer.apply_gradients( zip(gradients, v) ) # ADAM
    
    # for prediction
    train_pred = tf.nn.sigmoid(train.logits)
    
    # Saver
    saver = tf.train.Saver()

# 4. Train Energy predictor model

In [ ]:
reload_pos= False
load_pos_file=  working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_pos.ckpt"
    
reload_neg= False
load_neg_file= working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_neg.ckpt"

reload_all= True
load_all_file=  working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run-1) +"r_All"+comment+".ckpt";

save_all= True
save_all_file=  working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run) +"r_All"+comment+".ckpt";

In [ ]:
if current_run==1:
    reload_all= False;
else:
    reload_pos= False;
    reload_neg= False;

In [ ]:
num_steps = 3000 
summary_frequency = 50
n_valid_tests = 50
n_tests = 50
n_characters_step= num_unrollings*batch_size

train_error= list()
valid_error= list()

aux_print=0
with tf.Session(graph=graph) as session:
    # -------------------------------- Load weigths from file, or initialize variables ------------------------------------
    if reload_all:
        saver.restore(session, load_all_file)
        #session.run( global_step.assign(0) ) # SGD
        
    elif (reload_pos and reload_neg):
        tf.initialize_all_variables().run()
        pos_net.saver.restore(session, load_pos_file)
        neg_net.saver.restore(session, load_neg_file)
        print('Weights for positive and negative networks loaded')
    else:
        tf.initialize_all_variables().run()
        print('Weights Initialized')
    
    # ------------------------------------------- Training loop --------------------------------------------------
    print('step | Train_err | Valid_err | loss')
    
    mean_loss = 0.0
    mean_error = 0.0
    for step in range(num_steps):
        # 1. Get next batch
        batch_X, batch_y = dataset.train.next_batch()
        
        # 2. Setup feed dictionary for network 1 and network 2
        feed_dict = dict()
        # For dropout
        feed_dict[drop_prob] = dropout_cons
        # hyperparameters
        feed_dict[train.alpha_r] = alpha
        feed_dict[train.beta_r] = beta
            
        # Inroduce labels
        feed_dict[train.labels] = batch_y
        # Introduce inputs
        for i in range(num_unrollings+1):
            feed_dict[train.X[i]] = batch_X[i]
              
        # 3 Run optimizer
        #_, l, lr = session.run( [optimizer, loss_train, learning_rate], feed_dict=feed_dict) # SGD
        _, l, pred_train_aux = session.run( [optimizer, train.loss, train_pred], feed_dict=feed_dict) # ADAM
        
        mean_loss += l
        mean_error += classification_error(pred_train_aux, batch_y) 
        
        # --------------------------------------------- logging ------------------------------------------------
        if dataset.train.batch_generators[0]._text_size<aux_print :
            print('+'*80)
            print('')
            aux_print = 0
        else:
            aux_print += n_characters_step
        
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                mean_error = mean_error / summary_frequency
                        
            # ---------- Print loss in validation dataset ---------
            if allow_valid:
                ''' classification error on validation dataset '''
                mean_error_val = 0
                for step_valid in range(n_valid_tests):
                    # 1. Get next batch
                    batch_X_val, batch_y_val = dataset.valid.next_batch()        
                    feed_dict_val = dict()
                    # For dropout
                    feed_dict_val[drop_prob] = 1.0
                    # Inroduce labels
                    feed_dict_val[valid.labels] = batch_y_val
                    # Introduce inputs
                    for i in range(num_unrollings_val+1):
                        feed_dict_val[valid.X[i]] = batch_X_val[i]
                    # 2. Get prediction
                    prediction_val = session.run( [tf.nn.sigmoid(valid.logits)], feed_dict=feed_dict_val)          
                    mean_error_val = mean_error_val*step_valid
                    mean_error_val += classification_error(prediction_val, batch_y_val)
                    mean_error_val = mean_error_val/(step_valid+1)
                                    
                ''' print information '''
                train_error.append(mean_error)
                valid_error.append(mean_error_val)
                
                print('%d | %f | %f | %f ' % (step, 
                                              mean_error, 
                                              mean_error_val, 
                                              mean_loss
                                             ))    
                
                
                if(mean_error_val < 0.19):
                    break
            else:
                train_error.append(mean_error)
                print('%d | %f | %f' % (step, mean_error, mean_loss))
                         
            mean_loss = 0
            mean_error = 0
            
    if save_all:
        save_path = saver.save(session, save_all_file)
        print("LSTM cell Model for negative reviews saved in file: %s" % save_path)
    
    
    ''' classification error on test dataset '''        
    if allow_test:
        mean_error_test = 0
        for step_test in range(n_tests):
            # 1. Get next batch
            batch_X_test, batch_y_test = dataset.test.next_batch()        
            feed_dict_test = dict()
            # For dropout
            feed_dict_test[drop_prob] = 1.0
            # Inroduce labels
            feed_dict_test[test.labels] = batch_y_test
            # Introduce inputs
            for i in range(num_unrollings_test+1):
                feed_dict_test[test.X[i]] = batch_X_test[i]
            # 2. Get prediction
            prediction_test = session.run( [tf.nn.sigmoid(test.logits)], feed_dict=feed_dict_test)          
            mean_error_test = mean_error_test*step_test
            mean_error_test += classification_error(prediction_test, batch_y_test)
            mean_error_test = mean_error_test/(step_test+1)
            
        print("Classification error on test dataset:", mean_error_test)
        
    ''' perplexity '''
    
    
    

In [ ]:
%matplotlib inline

train_error_a = np.array(train_error)
valid_error_a = np.array(valid_error)

plt.plot(np.linspace(0, train_error_a.shape[0], train_error_a.shape[0]) , train_error_a )
plt.plot(np.linspace(0, valid_error_a.shape[0], train_error_a.shape[0]) , valid_error_a )

## Generate some samples from positive and negative networks

In [ ]:
with tf.Session(graph=graph) as session:
    # Load weigths from file, or initialize variables
    tf.initialize_all_variables().run()
    saver.restore(session, save_all_file)
    print('Weights loaded')
        
    # print values for b and w
    b_t= b.eval()
    w_t= w.eval()
    #b_t= session.run(b)
    print('b=', b_t)
    print('w=', w_t)
    
    
    # For positive network
    print('a' + '-'*30 + 'pos' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)
        pos_gen_test.reset_saved_out_state.run()
        for _ in range(30):
            prediction = tf.nn.softmax(pos_gen_test.y).eval({pos_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)
        print(sentence)
    print('a' + '-' * 80)
    
    # For negative network
    print('a' + '-'*30 + 'neg' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)
        neg_gen_test.reset_saved_out_state.run()
        for _ in range(30):
            prediction = tf.nn.softmax(neg_gen_test.y).eval({neg_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)
        print(sentence)
    print('a' + '-' * 80)